# Initiate GPU runtime

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Download sequencing run - Using Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

!ls gdrive/MyDrive

# Download sequencing run - Using MEGA.nz

In [ ]:
%%shell

sudo apt-get install -y megatools

# specify the MEGA url of the sequencing run (*.zip)
FILE=""

# specify the name of the folder (e. g. IGM_PVM_MINION_LIG_LIBRARYyyyymmdd)
FNAME=""

megadl "$FILE" --path /content/
unzip "$FNAME".zip; rm "$FNAME".zip

# Install Dorado (Oxford Nanopore Technologies)

In [ ]:
%%shell

# https://nanoporetech.com/software/other/dorado
DORADO_VERSION=1.0.0

wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-"$DORADO_VERSION"-linux-x64.tar.gz -O dorado.tar.gz
tar -vzxf dorado.tar.gz; rm -rf dorado.tar.gz; mv dorado-"$DORADO_VERSION"-linux-x64 ont-dorado; mkdir -p ont-dorado/models

./ont-dorado/bin/dorado download --directory ont-dorado/models

# BASECALL and DEMULTIPLEX process

In [ ]:
%%shell

# specify the name of the folder
FNAME=""

# specify the basecalling model(s) accuracy(ies): fast,hac,sup
# https://nanoporetech.com/platform/accuracy
ACCURACY=

echo "$ACCURACY" | tr ',' '\n' | while read i; do
    ./ont-dorado/bin/dorado basecaller --models-directory /content/ont-dorado/models/ "$i" \
      /content/"$FNAME"/ -r -v -x cuda:all --min-qscore 10 \
      -o /content/"$FNAME"-BASECALL-"$i"

    ./ont-dorado/bin/dorado summary /content/"$FNAME"-BASECALL-"$i"/*.bam \
      >> /content/"$FNAME"-BASECALL-"$i"/basecalling_summary.txt

    ./ont-dorado/bin/dorado demux /content/"$FNAME"-BASECALL-"$i" \
    -t $(nproc) -r --kit-name EXP-NBD114 --emit-summary \
    --output-dir /content/"$FNAME"-DEMUX-"$i"

    tar -czvf "$FNAME"-BASECALL-"$i".tar.gz "$FNAME"-BASECALL-"$i"
    tar -czvf "$FNAME"-DEMUX-"$i".tar.gz "$FNAME"-DEMUX-"$i"
done

# Download the BASECALL and DEMUX analysis (*.taz.gz)

In [1]:
from google.colab import files
import glob as gl

for i in gl.glob('*.tar.gz'):
    files.download(i)